In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
#################################################################
def Vh2V3k(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    v3k = Vh-25.2*cosl*cosb-245.7*sinl*cosb+276.8*sinb

    return v3k

#################################################################

def Vcmb2Vmod(Vcmb, omegam=0.27):
    
    omegal=1.-omegam
    c=299800.
    z=Vcmb/c
    q0=0.5*(omegam-2.*omegal)
    fmod=1.+0.5*(1.-q0)*z-(1./6)*(1.-q0-3.*q0**2+1.)*z**2
    Vmod=c*z*fmod

    return Vmod
#################################################################

def Vh2Vls(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    vls = Vh-26.*cosl*cosb+317.*sinl*cosb-8.*sinb

    return vls
#################################################################

In [3]:
df = pd.read_csv('6dfgs.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

print(len(df))

ln10 = np.log(10.)
df['DM_6dfgs']  = (5*np.log10(df["D75"])+25.)
df['eDM_6dfgs'] = 5.*df["feD"]/ln10
df_6dfgs = df.rename(columns={"Nest": "Nest_6dfgs"})
df_6dfgs = df_6dfgs.rename(columns={"Vcmb": "Vcmb_6dfgs"})
df_6dfgs = df_6dfgs.rename(columns={"PGC1": "PGC1_6dfgs"})

df_6dfgs  = df_6dfgs[df_6dfgs['rej']==0]

df_6dfgs.head()

8885


,rej,PGC1_6dfgs,Nest_6dfgs,Name,Vcmb_6dfgs,Vgp,D75,D100,feD,DzDh,eDz,skew,RA,DE,Vg,Gp,DM_6dfgs,eDM_6dfgs
PGC,,,,,,,,,,,,,,,,,,
64,0,72642,200033,g0000523-355037,15324,14646,222.3,166.7,0.29,-0.034659,0.110224,0.010894,0.2181,-35.8437,14646,1261,36.734697,0.629727
66,0,72642,200033,g0000532-355911,14725,14646,191.0,143.2,0.27,0.030506,0.103811,-0.028422,0.2216,-35.9863,14646,1261,36.405167,0.586298
114,0,72642,200033,g0001341-361900,14397,14397,133.6,100.2,0.27,0.177860,0.105257,-0.103135,0.3923,-36.3167,14397,-1,35.629032,0.586298
115,0,115,0,g0001453-042049,14005,14005,188.5,141.4,0.27,0.015661,0.103465,-0.020551,0.4387,-4.3469,14005,-1,36.376557,0.586298
123,0,123,205549,g0001361-144455,10966,10966,160.2,120.1,0.26,-0.023948,0.099643,0.003134,0.4002,-14.7487,10966,-1,36.023313,0.564583


In [4]:
len(df_6dfgs)

7112

In [5]:
df = pd.read_csv('FP_sdss_v3.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

ln10 = np.log(10.)
df_fpsdss = df.rename(columns={"Nest": "Nest_fpsdss"})
df_fpsdss = df_fpsdss.rename(columns={"cz": "Vcmb_fpsdss"})
df_fpsdss = df_fpsdss.rename(columns={"1PGC": "PGC1_fpsdss"})

# Dc = Dist / Dist_fit
# Dist_fid = cz/H0  , H0 = 75

# 
# df_fpsdss["eDM_fpsdss"]

logDist = df_fpsdss.logDc + np.log10(df_fpsdss.Vcmb_fpsdss/75.)
df_fpsdss["DM_fpsdss"] = 5. * logDist + 25.
df_fpsdss["eDM_fpsdss"] = 5. * df_fpsdss.elogDc


df_fpsdss.head()

,logD,elogD,skew,logDc,elogDc,Vcmb_fpsdss,gcz,V3kg,V3kt14,Nest_fpsdss,...,In,Bar_N,M,jrl,pel,nvot,objid,specid,DM_fpsdss,eDM_fpsdss
PGC,,,,,,,,,,,,,,,,,,,,,
20919,0.033960,0.096999,-0.606800,-0.017410,0.122226,16176,16144,16178,16143,104087,...,1,0.002052,0,1,0.83,23,1237663547968323716,1951253584033114112,36.581999,0.611130
21014,-0.193441,0.098885,-0.230652,-0.269684,0.124203,3248,3250,3247,3250,111419,...,1,0.000727,2,1,0.62,29,1237663917871988832,2099918005914855424,31.834354,0.621015
21072,-0.002285,0.095861,-0.599287,-0.048528,0.120334,14976,15017,14974,15016,104427,...,1,0.002429,2,1,1.00,22,1237663531326243079,1951295365474969600,36.259033,0.601670
21226,-0.009975,0.095778,-0.598648,-0.083456,0.120682,16281,16283,17237,16282,101391,...,1,0.002031,2,1,0.57,47,1237653588476952852,609190772427745280,36.265819,0.603410
21354,0.054988,0.110874,-0.667778,0.017312,0.139151,26328,26084,26084,26014,0,...,1,0.001211,0,1,0.94,35,1237673703967031397,486471804492212224,37.813343,0.695755


In [6]:
df = pd.read_csv('all_cf2_fp.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_cf2fp = df.set_index('PGC')

for suffix in ["smc","enr","far"]:
    df_cf2fp = df_cf2fp.rename(columns={"DM"+suffix: "DM_"+suffix})
    df_cf2fp["eDM_"+suffix] = 0.50
    df_cf2fp["DM_"+suffix][df_cf2fp["DM_"+suffix]==0] = np.nan
    df_cf2fp["eDM_"+suffix][df_cf2fp["DM_"+suffix].isna()] = np.nan
    df_cf2fp["PGC1_"+suffix] = df_cf2fp["PGC1"]
    df_cf2fp["Nest_"+suffix] = df_cf2fp["Nest"]
    df_cf2fp["Vcmb_"+suffix] = df_cf2fp["Vcmb"]


del df_cf2fp['PGC1']
del df_cf2fp['Nest']
del df_cf2fp['Vcmb']


<ipython-input-6-cc81d93aba88>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf2fp["DM_"+suffix][df_cf2fp["DM_"+suffix]==0] = np.nan
<ipython-input-6-cc81d93aba88>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf2fp["eDM_"+suffix][df_cf2fp["DM_"+suffix].isna()] = np.nan
<ipython-input-6-cc81d93aba88>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf2fp["DM_"+suffix][df_cf2fp["DM_"+suffix]==0] = np.nan
<ipython-input-6-

In [7]:
len(df_6dfgs)

7112

In [8]:
how = 'outer'

df = df_6dfgs.join(df_fpsdss, lsuffix='_6dfgs', rsuffix='_fpsdss', how=how)

df = df.join(df_cf2fp, lsuffix='_fp', rsuffix='_cf2fp', how=how)

# df1 =  df[['PGC1_far','eDM_far']]
# df1g = df1.groupby('PGC1_far').count().rename(columns={"eDM_far":"N_far"})
# df1 = df1.reset_index().set_index('PGC1_far').join(df1g, lsuffix='_l', rsuffix='_r', how='left').reset_index().set_index('PGC')
# df1['eDM_far'] = df1['eDM_far']/np.sqrt(df1["N_far"])

# df['eDM_far'] = df1['eDM_far']
# df["N_far"] = df1["N_far"]


for col in df.columns:
    if col.split('_')[0]=='PGC1' and col!='PGC1_fpsdss':
        df['PGC1_fpsdss'] = df['PGC1_fpsdss'].fillna(df[col])
        
for col in df.columns:
    if col.split('_')[0]=='Nest' and col!='Nest_fpsdss':
        df['Nest_fpsdss'] = df['Nest_fpsdss'].fillna(df[col])



catalogs = ["fpsdss", "6dfgs", "smc", "enr", "far"]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat, 'Nest_'+cat, "DM_"+cat, "eDM_"+cat, "Vcmb_"+cat]

df = df[ss]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat]   
df['pgc1'] = df[ss].median(axis=1)
df = df[~df['pgc1'].isna()]


print(len(df))

df.head()

38370


,PGC1_fpsdss,Nest_fpsdss,DM_fpsdss,eDM_fpsdss,Vcmb_fpsdss,PGC1_6dfgs,Nest_6dfgs,DM_6dfgs,eDM_6dfgs,Vcmb_6dfgs,...,Nest_enr,DM_enr,eDM_enr,Vcmb_enr,PGC1_far,Nest_far,DM_far,eDM_far,Vcmb_far,pgc1
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,NaN,NaN,NaN,72642.0,200033.0,36.734697,0.629727,15324.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72642.0
66,72642.0,200033.0,NaN,NaN,NaN,72642.0,200033.0,36.405167,0.586298,14725.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72642.0
114,72642.0,200033.0,NaN,NaN,NaN,72642.0,200033.0,35.629032,0.586298,14397.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72642.0
115,115.0,0.0,NaN,NaN,NaN,115.0,0.0,36.376557,0.586298,14005.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.0
123,123.0,205549.0,NaN,NaN,NaN,123.0,205549.0,36.023313,0.564583,10966.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0


In [9]:
for cat in catalogs:
    df['Vmod_'+cat] = Vcmb2Vmod(df['Vcmb_'+cat])
    logD = (df['DM_'+cat]-25)/5.
    df['logH_'+cat] = np.log10(df['Vmod_'+cat]) - logD 
    
    print(cat, 10**df['logH_'+cat][df['Vmod_'+cat]>4000].median())

fpsdss 81.28175524959163
6dfgs 72.56945650701134
smc 76.73704799488011
enr 76.22256166035713
far 80.36152475092275


In [10]:
sigmaClips = {}

for i in range(1):
    for cat in catalogs:

        logH = df['logH_'+cat]
        med = logH[df['Vmod_'+cat]>4000].median()
        stdev = logH[df['Vmod_'+cat]>4000].std()
        
#         print cat, 10**med
        
        if not cat in sigmaClips:
                sigmaClips[cat] = []
        sigmaClips[cat] += list(df[(logH > med+3.5*stdev) | (logH < med-3.5*stdev)].index.values)
        df['DM_'+cat][(logH > med+3.5*stdev) | (logH < med-3.5*stdev)] = np.nan
        df['eDM_'+cat][(logH > med+3.5*stdev) | (logH < med-3.5*stdev)] = np.nan
        df['logH_'+cat][(logH > med+3.5*stdev) | (logH < med-3.5*stdev)] = np.nan


In [11]:
myDict = {}
myDict['fpsdss'] = 0

for cat in catalogs[1:]:

        delta = df['DM_fpsdss'] - df['DM_'+cat]

        stdev = delta.std()
        med   = delta.median()
        myDict[cat] = med


In [12]:
for cat in sigmaClips:
    print(cat, len(sigmaClips[cat]))

myDict

fpsdss 79
6dfgs 13
smc 1
enr 2
far 0


{'fpsdss': 0,
 '6dfgs': -0.5304647349858769,
 'smc': -0.5211074177986177,
 'enr': -0.5019526537537828,
 'far': -0.17085862314882405}

In [13]:
ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
print(len(df["PGC1_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
    df["PGC1_fpsdss"][ind] = df["PGC1_"+cat][ind]
    

ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
print(len(df["PGC1_fpsdss"][ind]))

ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
df["PGC1_fpsdss"][ind] = np.nan

22040
22020


In [14]:
ind = (df["Nest_fpsdss"].isna()) | (df["Nest_fpsdss"]==0)
print(len(df["Nest_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["Nest_fpsdss"].isna()) | (df["Nest_fpsdss"]==0)
    df["Nest_fpsdss"][ind] = df["Nest_"+cat][ind]
    
ind = (df["Nest_fpsdss"].isna()) | (df["Nest_fpsdss"]==0)
df["Nest_fpsdss"][ind] = np.nan

24616


In [15]:
ind = (df["Vmod_fpsdss"].isna()) | (df["Vmod_fpsdss"]==0)
print(len(df["Vmod_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["Vmod_fpsdss"].isna()) | (df["Vmod_fpsdss"]==0)
    df["Vmod_fpsdss"][ind] = df["Vmod_"+cat][ind]
    
ind = (df["Vmod_fpsdss"].isna()) | (df["Vmod_fpsdss"]==0)
df["Vmod_fpsdss"][ind] = np.nan

8218


In [16]:
ind = (df["Vcmb_fpsdss"].isna()) | (df["Vcmb_fpsdss"]==0)
print(len(df["Vcmb_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["Vcmb_fpsdss"].isna()) | (df["Vcmb_fpsdss"]==0)
    df["Vcmb_fpsdss"][ind] = df["Vcmb_"+cat][ind]
    
ind = (df["Vcmb_fpsdss"].isna()) | (df["Vcmb_fpsdss"]==0)
df["Vcmb_fpsdss"][ind] = np.nan

8218


In [17]:
df.head()

,PGC1_fpsdss,Nest_fpsdss,DM_fpsdss,eDM_fpsdss,Vcmb_fpsdss,PGC1_6dfgs,Nest_6dfgs,DM_6dfgs,eDM_6dfgs,Vcmb_6dfgs,...,Vmod_fpsdss,logH_fpsdss,Vmod_6dfgs,logH_6dfgs,Vmod_smc,logH_smc,Vmod_enr,logH_enr,Vmod_far,logH_far
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,NaN,NaN,15324.0,72642.0,200033.0,36.734697,0.629727,15324.0,...,15938.430745,NaN,15938.430745,1.855506,NaN,NaN,NaN,NaN,NaN,NaN
66,72642.0,200033.0,NaN,NaN,14725.0,72642.0,200033.0,36.405167,0.586298,14725.0,...,15292.703769,NaN,15292.703769,1.903451,NaN,NaN,NaN,NaN,NaN,NaN
114,72642.0,200033.0,NaN,NaN,14397.0,72642.0,200033.0,35.629032,0.586298,14397.0,...,14939.887452,NaN,14939.887452,2.048541,NaN,NaN,NaN,NaN,NaN,NaN
115,115.0,NaN,NaN,NaN,14005.0,115.0,0.0,36.376557,0.586298,14005.0,...,14518.945116,NaN,14518.945116,1.886624,NaN,NaN,NaN,NaN,NaN,NaN
123,123.0,205549.0,NaN,NaN,10966.0,123.0,205549.0,36.023313,0.564583,10966.0,...,11282.137786,NaN,11282.137786,1.847729,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
myDict = {}
myDict['fpsdss'] = 0

for cat in catalogs[1:]:

        delta = df['DM_fpsdss'] - df['DM_'+cat]

        stdev = delta.std()
        med   = delta.median()
        myDict[cat] = med


In [19]:
for cat in sigmaClips:
    print(cat, len(sigmaClips[cat]))

myDict

fpsdss 79
6dfgs 13
smc 1
enr 2
far 0


{'fpsdss': 0,
 '6dfgs': -0.5304647349858769,
 'smc': -0.5211074177986177,
 'enr': -0.5019526537537828,
 'far': -0.17085862314882405}

In [20]:
cat = catalogs[0]
N = len(sigmaClips[cat])
x = [1 for i in range(N)]
Dreject = pd.DataFrame(list(zip(sigmaClips[cat], x)), columns=['PGC',cat])
Dreject = Dreject.set_index('PGC')

for cat in catalogs[1:]:
    x = [1 for i in range(N)]
    Dj = pd.DataFrame(list(zip(sigmaClips[cat], x)), columns=['PGC',cat])
    Dj = Dj.set_index('PGC')
    Dreject = Dreject.join(Dj, how='outer')

Dreject = Dreject.fillna(0).astype('int')
Dreject = Dreject.reset_index()
Dreject.head(20)

import pandas as pd
from tabulate import tabulate

content = tabulate(Dreject.values.tolist(), list(Dreject.columns), tablefmt="pipe")
open('3SigmaClips_FP.csv', "w").write(content)

5528

In [21]:
sig3 = pd.read_csv('3SigmaClips_FP.csv', delimiter='|')
sig3 = sig3.rename(columns=lambda x: x.strip())
new_col = {}
for cat in catalogs:
    new_col[cat] = cat+'_flag'

sig3 = sig3.drop(index=0).set_index('PGC')[catalogs]
sig3 = sig3.rename(columns=new_col)
sig3 = sig3.astype('int32')
sig3.index = sig3.index.astype('int32')

sig3.head(10)

,fpsdss_flag,6dfgs_flag,smc_flag,enr_flag,far_flag
PGC,,,,,
15472,0,1,0,0,0
31765,0,0,0,1,0
40653,0,0,0,1,0
49131,0,0,1,0,0
66351,0,1,0,0,0
117059,1,0,0,0,0
160575,0,1,0,0,0
185183,0,1,0,0,0
218564,0,1,0,0,0


In [22]:
len(sig3[sig3["fpsdss_flag"]==1])

79

In [23]:
df.index = df.index.astype('int32')
df = df.join(sig3, lsuffix='_l', rsuffix='_r', how="outer")

for cat in catalogs:
    ind = df[cat+"_flag"].isna()
    df[cat+"_flag"][ind] = 0

df.head()

,PGC1_fpsdss,Nest_fpsdss,DM_fpsdss,eDM_fpsdss,Vcmb_fpsdss,PGC1_6dfgs,Nest_6dfgs,DM_6dfgs,eDM_6dfgs,Vcmb_6dfgs,...,logH_smc,Vmod_enr,logH_enr,Vmod_far,logH_far,fpsdss_flag,6dfgs_flag,smc_flag,enr_flag,far_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,NaN,NaN,15324.0,72642.0,200033.0,36.734697,0.629727,15324.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
66,72642.0,200033.0,NaN,NaN,14725.0,72642.0,200033.0,36.405167,0.586298,14725.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
114,72642.0,200033.0,NaN,NaN,14397.0,72642.0,200033.0,35.629032,0.586298,14397.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
115,115.0,NaN,NaN,NaN,14005.0,115.0,0.0,36.376557,0.586298,14005.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
123,123.0,205549.0,NaN,NaN,10966.0,123.0,205549.0,36.023313,0.564583,10966.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [24]:
myDict = {}
myDict["fpsdss"] = 0
myDict["6dfgs"] = -0.358
myDict["smc"] = -0.213
myDict["enr"] = -0.238
myDict["far"] = -0.84


for cat in catalogs:
    df['DM_'+cat] += myDict[cat]
    df['w_'+cat] = (1.-df[cat+'_flag'])/df['eDM_'+cat]**2
    df['w_'+cat] = df['w_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['w_'+cat] = df.apply(lambda x: x['w_'+cat] if x[cat+'_flag']==0 else np.nan, axis=1)
    df['e_'+cat] = (1.-df[cat+'_flag'])*df['eDM_'+cat]
    df['e_'+cat] = df['e_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['e_'+cat] = df.apply(lambda x: x['e_'+cat] if x[cat+'_flag']==0 else np.nan, axis=1)
    df['xw_'+cat] = (df['DM_'+cat])*df['w_'+cat]
    df['xw_'+cat] = df.apply(lambda x: x['xw_'+cat] if x[cat+'_flag']==0 else np.nan, axis=1)

sw = ['w_'+cat for cat in catalogs]
ee = ['e_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)

df["eDM_av"] = df[ee].min(axis=1) # np.sqrt(Err2)
df["DM_av"] = df[sx].sum(axis=1)*Err2   

In [25]:
df['av_flag'] = 0*df['DM_av']
df['av_flag'][df['DM_av'].isna()] = 1

In [26]:
for cat in catalogs:
    df['w_'+cat] = 1./df['eDM_'+cat]**2
    df['xw_'+cat] = df['DM_'+cat]*df['w_'+cat]
    df['e_'+cat] = 1.*df['eDM_'+cat]

sw = ['w_'+cat for cat in catalogs]
ee = ['e_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)
df["DM_av_"] = df[sx].sum(axis=1)*Err2
df["eDM_av_"] = df[ee].min(axis=1)  #  np.sqrt(Err2)

ind = df["av_flag"] == 1
df["eDM_av"][ind] = df["eDM_av_"][ind]
df["DM_av"][ind] = df["DM_av_"][ind]

In [27]:
df[ee+["eDM_av"]].head()

,e_fpsdss,e_6dfgs,e_smc,e_enr,e_far,eDM_av
PGC,,,,,,
64,NaN,0.629727,NaN,NaN,NaN,0.629727
66,NaN,0.586298,NaN,NaN,NaN,0.586298
114,NaN,0.586298,NaN,NaN,NaN,0.586298
115,NaN,0.586298,NaN,NaN,NaN,0.586298
123,NaN,0.564583,NaN,NaN,NaN,0.564583


In [28]:
ss = ['PGC1_fpsdss', 'Nest_fpsdss', 'Vcmb_fpsdss', 'Vmod_fpsdss', "DM_av", "eDM_av", "av_flag"]
for cat in catalogs:
    ss += ["DM_"+cat, "eDM_"+cat, cat+'_flag']

df = df.fillna(0)

df = df[ss]


df = df.rename(columns={'PGC1_fpsdss': 'PGC1',
                        'Nest_fpsdss': 'Nest',
                        'Vcmb_fpsdss': 'Vcmb',
                        'Vmod_fpsdss': 'Vmod'
                       })

df.head()

,PGC1,Nest,Vcmb,Vmod,DM_av,eDM_av,av_flag,DM_fpsdss,eDM_fpsdss,fpsdss_flag,...,6dfgs_flag,DM_smc,eDM_smc,smc_flag,DM_enr,eDM_enr,enr_flag,DM_far,eDM_far,far_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,15324.0,15938.430745,36.376697,0.629727,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,72642.0,200033.0,14725.0,15292.703769,36.047167,0.586298,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,72642.0,200033.0,14397.0,14939.887452,35.271032,0.586298,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,115.0,0.0,14005.0,14518.945116,36.018557,0.586298,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,123.0,205549.0,10966.0,11282.137786,35.665313,0.564583,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
np.max(df.DM_fpsdss.values)

40.26363120292504

In [30]:
for cat in catalogs:
    edm = 'eDM_'+cat
    dm = 'DM_'+cat
    flag = cat+'_flag'
    
    print(dm, edm, flag)
    
    df[edm] = df.apply(lambda x: x[edm] if x[dm]!=0 else np.nan, axis=1)
    df[flag] = df.apply(lambda x: x[flag] if x[dm]!=0 else np.nan, axis=1)
    df[dm] = df.apply(lambda x: x[dm] if x[dm]!=0 else np.nan, axis=1)

DM_fpsdss eDM_fpsdss fpsdss_flag
DM_6dfgs eDM_6dfgs 6dfgs_flag
DM_smc eDM_smc smc_flag
DM_enr eDM_enr enr_flag
DM_far eDM_far far_flag


In [31]:
df.head(20)

,PGC1,Nest,Vcmb,Vmod,DM_av,eDM_av,av_flag,DM_fpsdss,eDM_fpsdss,fpsdss_flag,...,6dfgs_flag,DM_smc,eDM_smc,smc_flag,DM_enr,eDM_enr,enr_flag,DM_far,eDM_far,far_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,15324.0,15938.430745,36.376697,0.629727,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,72642.0,200033.0,14725.0,15292.703769,36.047167,0.586298,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,72642.0,200033.0,14397.0,14939.887452,35.271032,0.586298,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,115.0,0.0,14005.0,14518.945116,36.018557,0.586298,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,123.0,205549.0,10966.0,11282.137786,35.665313,0.564583,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,75.0,200697.0,11615.0,11969.416136,35.610405,0.564583,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,72642.0,200033.0,14365.0,14905.495569,36.359043,0.608012,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,382.0,200108.0,10147.0,10417.919123,35.710916,0.542868,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,262.0,208331.0,8795.0,8998.830832,35.213386,0.629727,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df.to_csv("FP_CF4_components_bar.csv", sep='|')

In [33]:
table   = np.genfromtxt('FP_CF4_components_bar.csv' , delimiter='|', 
                        filling_values=-10000, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["PGC", "PGC1", "Nest"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]+["av_flag","fpsdss_flag","6dfgs_flag","smc_flag","sfi_flag","enr_flag","far_flag"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.2f'))

## to be used on EDD
myTable.write('FP_CF4_components_bar', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)